# Exploring the Impact of the Lionesses' Success on the Women's Super League (WSL)
## data_preparation.ipynb — Data Loading & Cleaning
To load all WSL-related data, perform basic cleaning, and merge the datasets into a single clean file ready for analysis.

#### Set-up, Import and Config

In [1]:
# Imports
import sys
import os
from pathlib import Path
import pandas as pd
import re

# Working directory
current = Path.cwd()
for parent in [current] + list(current.parents):
    if (parent / ".git").exists():
        repo_root = parent
        break
sys.path.append(str(repo_root))

# Function Imports
from config.general_config import DATA_IN, DATA_OUT
from config.projects.wsl_config import SEASON_COL_MAP, NATIONALITY_COL_MAP, CLUB_NAME_MAP, WSL_DROP_COLUMNS, NATIONALITY_DROP_COLUMNS
from src.data_utils import load_csv_data, drop_columns
from src.wsl_utils import process_wsl_data, process_nationality_data

In [2]:
# Config
SEASONS = range(2017, 2025)
SEASON_FILE_NAME_TEMPLATE = 'wsl_{start}_{end}.csv'
NATIONALITY_FILE_NAME_TEMPLATE = 'nationality_table - {start}-{end}.csv'

## WSL Season Data 
#### Import and Data Cleaning
The following cleaning steps will be done:
* String cleaning to remove 'Club Crest ' text
* Normalising club names to be a single string
* Add a season column to each data set to be used for unique identifiers and for clearer season rankings
* Column splitting, to separate the top goal scorer and the number of goals scored
* Renaming columns to more meaningful and machine learning friendly options
* Mapping categorical info to new values
* Converting column data types

In [3]:
# Data Import
for start_year in SEASONS:
    end_year = start_year + 1
    file_name = SEASON_FILE_NAME_TEMPLATE.format(start=start_year, end=end_year)

    df = load_csv_data(DATA_IN / file_name)

    var_name = f'wsl_{start_year}_{end_year}'

    print(f'Loaded {file_name} as df name: {var_name}')

    # Create a variable for each season dynamically
    globals()[var_name] = df

Loaded wsl_2017_2018.csv as df name: wsl_2017_2018
Loaded wsl_2018_2019.csv as df name: wsl_2018_2019
Loaded wsl_2019_2020.csv as df name: wsl_2019_2020
Loaded wsl_2020_2021.csv as df name: wsl_2020_2021
Loaded wsl_2021_2022.csv as df name: wsl_2021_2022
Loaded wsl_2022_2023.csv as df name: wsl_2022_2023
Loaded wsl_2023_2024.csv as df name: wsl_2023_2024
Loaded wsl_2024_2025.csv as df name: wsl_2024_2025


In [4]:
# Look at a dataset (latest season) as each set follows the same pattern
wsl_2024_2025.head(2)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Club Crest Chelsea,22,19,3,0,56,13,43,60,2.73,48.2,19.2,28.9,1.32,"9,373",Agnes Beever-Jones - 9,Hannah Hampton,NaN
1,2,Club Crest Arsenal,22,15,3,4,62,26,36,48,2.18,47.9,20.5,27.4,1.25,"28,808",Alessia Russo - 12,Daphne van Domselaar,NaN


In [5]:
# Shape of each dataset before cleaning
print(f' WSL Data Shape (Season Order (2017 to 2025)): {wsl_2017_2018.shape}, {wsl_2018_2019.shape}, {wsl_2019_2020.shape}, {wsl_2020_2021.shape}, {wsl_2021_2022.shape}, {wsl_2022_2023.shape}, {wsl_2023_2024.shape}, {wsl_2024_2025.shape}')

 WSL Data Shape (Season Order (2017 to 2025)): (10, 15), (11, 19), (12, 19), (12, 19), (12, 19), (12, 19), (12, 19), (12, 19)


In [6]:
for start_year in SEASONS:
    end_year = start_year + 1
    var_name = f"wsl_{start_year}_{end_year}"

    df = globals().get(var_name)

    cleaned_df = process_wsl_data(df, start_year, end_year)

    globals()[var_name] = cleaned_df

Cleaning WSL for season 2017_2018...
Finished cleaning 2017_2018 data.
Cleaning WSL for season 2018_2019...
Finished cleaning 2018_2019 data.
Cleaning WSL for season 2019_2020...
Finished cleaning 2019_2020 data.
Cleaning WSL for season 2020_2021...
Finished cleaning 2020_2021 data.
Cleaning WSL for season 2021_2022...
Finished cleaning 2021_2022 data.
Cleaning WSL for season 2022_2023...
Finished cleaning 2022_2023 data.
Cleaning WSL for season 2023_2024...
Finished cleaning 2023_2024 data.
Cleaning WSL for season 2024_2025...
Finished cleaning 2024_2025 data.


In [7]:
# Look at the same dataset after cleaning
wsl_2024_2025.head(2)

,Rank,Club,Matches,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points,...,Expected_Goals_Difference,Expected_Goals_Difference_Per_90_Mins,Attendance,Top_Scorer,Goalkeeper,Notes,season,unique_ID,Top_Scorer_Name,Top_Scorer_Goals
0,1,Chelsea,22,19,3,0,56,13,43,60,...,28.9,1.32,9373,Agnes Beever-Jones - 9,Hannah Hampton,NaN,2024_2025,Chelsea_2024_2025,Agnes_Beever-Jones,9
1,2,Arsenal,22,15,3,4,62,26,36,48,...,27.4,1.25,28808,Alessia Russo - 12,Daphne van Domselaar,NaN,2024_2025,Arsenal_2024_2025,Alessia_Russo,12


In [8]:
# Shape of each dataset after cleaning
print(f' WSL Data Shape (Season Order (2017 to 2025)): {wsl_2017_2018.shape}, {wsl_2018_2019.shape}, {wsl_2019_2020.shape}, {wsl_2020_2021.shape}, {wsl_2021_2022.shape}, {wsl_2022_2023.shape}, {wsl_2023_2024.shape}, {wsl_2024_2025.shape}')

 WSL Data Shape (Season Order (2017 to 2025)): (10, 19), (11, 23), (12, 23), (12, 23), (12, 23), (12, 23), (12, 23), (12, 23)


## Nationality Data
#### Import and Data Cleaning
The following cleaning steps will be done:
* Add a season column to each data set to be used for unique identifiers and for clearer season rankings
* Normalising country names to be a single string
* Renaming columns 
* Converting column data types
* Add nationality flag (England, European (excl. Eng), and Non-European)

In [9]:
# Data Import
for start_year in SEASONS:
    end_year = start_year + 1
    file_name = NATIONALITY_FILE_NAME_TEMPLATE.format(start=start_year, end=end_year)

    df = load_csv_data(DATA_IN / file_name)

    var_name = f'nationality_{start_year}_{end_year}'

    print(f'Loaded {file_name} as df name: {var_name}')

    # Create a variable for each season dynamically
    globals()[var_name] = df

Loaded nationality_table - 2017-2018.csv as df name: nationality_2017_2018
Loaded nationality_table - 2018-2019.csv as df name: nationality_2018_2019
Loaded nationality_table - 2019-2020.csv as df name: nationality_2019_2020
Loaded nationality_table - 2020-2021.csv as df name: nationality_2020_2021
Loaded nationality_table - 2021-2022.csv as df name: nationality_2021_2022
Loaded nationality_table - 2022-2023.csv as df name: nationality_2022_2023
Loaded nationality_table - 2023-2024.csv as df name: nationality_2023_2024
Loaded nationality_table - 2024-2025.csv as df name: nationality_2024_2025


In [10]:
# Look at a dataset (latest season) as each set follows the same pattern
nationality_2024_2025.head(2)

,Rk,Nation,# Players,Min,List,FIFA Ranking
0,1,eng England,126,"81,363","Alessia Russo, Nikita Parris, Rachel Daly, Gra...",4
1,2,nl Netherlands,17,"16,846","Kerstin Casparij, Katja Snoeijs, Dominique Jan...",10


In [11]:
# Shape of each dataset before cleaning
print(f' Nationality Data Shape (Season Order (2017 to 2025)): {nationality_2017_2018.shape}, {nationality_2018_2019.shape}, {nationality_2019_2020.shape}, {nationality_2020_2021.shape}, {nationality_2021_2022.shape}, {nationality_2022_2023.shape}, {nationality_2023_2024.shape}, {nationality_2024_2025.shape}')

 Nationality Data Shape (Season Order (2017 to 2025)): (20, 6), (26, 6), (28, 6), (34, 6), (36, 7), (36, 6), (34, 6), (34, 6)


In [12]:
# Data Cleaning
for start_year in SEASONS:
    end_year = start_year + 1
    var_name = f"nationality_{start_year}_{end_year}"

    df = globals().get(var_name)

    cleaned_df = process_nationality_data(df, start_year, end_year)

    globals()[var_name] = cleaned_df

Cleaning nationality season data for 2017_2018...
Finished cleaning nationality data for 2017_2018.
Cleaning nationality season data for 2018_2019...
Finished cleaning nationality data for 2018_2019.
Cleaning nationality season data for 2019_2020...
Finished cleaning nationality data for 2019_2020.
Cleaning nationality season data for 2020_2021...
Finished cleaning nationality data for 2020_2021.
Cleaning nationality season data for 2021_2022...
Finished cleaning nationality data for 2021_2022.
Cleaning nationality season data for 2022_2023...
Finished cleaning nationality data for 2022_2023.
Cleaning nationality season data for 2023_2024...
Finished cleaning nationality data for 2023_2024.
Cleaning nationality season data for 2024_2025...
Finished cleaning nationality data for 2024_2025.


In [13]:
# Look at the same dataset after cleaning
nationality_2024_2025.head(2)

,Rank,Nationality,Num_Players,Minutes_Played,List_of_Players,FIFA Ranking,season,Season_Rank,euro_flag,group
0,1,England,126,81363,"Alessia Russo, Nikita Parris, Rachel Daly, Gra...",4,2024_2025,2024_2025_1,1,English
1,2,Netherlands,17,16846,"Kerstin Casparij, Katja Snoeijs, Dominique Jan...",10,2024_2025,2024_2025_2,1,European (excl. Eng)


In [14]:
# Shape of each dataset after cleaning
print(f' Nationality Data Shape (Season Order (2017 to 2025)): {nationality_2017_2018.shape}, {nationality_2018_2019.shape}, {nationality_2019_2020.shape}, {nationality_2020_2021.shape}, {nationality_2021_2022.shape}, {nationality_2022_2023.shape}, {nationality_2023_2024.shape}, {nationality_2024_2025.shape}')

 Nationality Data Shape (Season Order (2017 to 2025)): (20, 10), (26, 10), (28, 10), (34, 10), (36, 11), (36, 10), (34, 10), (34, 10)


## Stadium Data
#### Import and Data Cleaning
The following cleaning steps will be done:

* Club names will be standardised
* Converting column data types

In [15]:
# Data Import
stadium_file_name = 'wsl_stadium_data.csv'
df_stadiums = load_csv_data(DATA_IN / stadium_file_name)

# Data Cleaning 
# Standardise names
df_stadiums["Team"] = (df_stadiums["Team"].str.strip().str.replace(" ", "_"))
# Apply mapping
df_stadiums["Team"] = df_stadiums["Team"].replace(CLUB_NAME_MAP)

In [16]:
# Save Data
df_stadiums.to_csv(DATA_OUT/ stadium_file_name, index=False)

## Data Merge
* Merge WSL Stats Tables
* Merge Nationality Tables

#### WSL Seasonal Data

In [17]:
# Create a list of Datasets for merging
season_dfs = [
    wsl_2017_2018,
    wsl_2018_2019,
    wsl_2019_2020,
    wsl_2020_2021,
    wsl_2021_2022,
    wsl_2022_2023,
    wsl_2023_2024,
    wsl_2024_2025,
]
wsl_combined = pd.concat(season_dfs, ignore_index=True)

In [18]:
# Look at the combined data
wsl_combined.head(2)

,Rank,Club,Matches,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points,...,Goalkeeper,Notes,season,unique_ID,Top_Scorer_Name,Top_Scorer_Goals,Expected_Goals,Expected_Goals_Allowed,Expected_Goals_Difference,Expected_Goals_Difference_Per_90_Mins
0,1,Chelsea,18,13,5,0,44,13,31,44,...,Hedvig Lindahl,→ Champions League via league finish,2017_2018,Chelsea_2017_2018,Fran_Kirby,8,NaN,NaN,NaN,NaN
1,2,Manchester_City,18,12,2,4,51,17,34,38,...,Ellie Roebuck,→ Champions League via league finish,2017_2018,Manchester_City_2017_2018,Nikita_Parris,11,NaN,NaN,NaN,NaN


In [19]:
# Drop specified columns
wsl_combined = drop_columns(wsl_combined, WSL_DROP_COLUMNS)

In [20]:
# Shape of combined data
wsl_combined.shape

(93, 18)

In [21]:
# Replace Season with just one value (convert "2017_2018" → 2017)
wsl_combined['Season'] = wsl_combined['season'].str.split('_').str[0].astype(int)
wsl_combined = wsl_combined.drop(columns=['season'])

In [22]:
# Look at the data after the transformation
wsl_combined

,Rank,Club,Matches,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points,Points_Per_Match,Attendance,Top_Scorer,unique_ID,Top_Scorer_Name,Top_Scorer_Goals,Expected_Goals_Difference_Per_90_Mins,Season
0,1,Chelsea,18,13,5,0,44,13,31,44,2.44,1884.0,Fran Kirby - 8,Chelsea_2017_2018,Fran_Kirby,8,NaN,2017
1,2,Manchester_City,18,12,2,4,51,17,34,38,2.11,3818.0,Nikita Parris - 11,Manchester_City_2017_2018,Nikita_Parris,11,NaN,2017
2,3,Arsenal,18,11,4,3,38,18,20,37,2.06,586.0,Beth Mead - 8,Arsenal_2017_2018,Beth_Mead,8,NaN,2017
3,4,Reading,18,9,5,4,40,18,22,32,1.78,647.0,Remi Allen - 9,Reading_2017_2018,Remi_Allen,9,NaN,2017
4,5,Birmingham_City,18,9,3,6,30,18,12,30,1.67,741.0,Ellen White - 15,Birmingham_City_2017_2018,Ellen_White,15,NaN,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,8,Everton,22,6,6,10,24,32,-8,24,1.09,1956.0,"Honoka Hayashi, Kelly Gago... - 4",Everton_2024_2025,"Honoka_Hayashi, Kelly_Gago...",4,-0.85,2024
89,9,West_Ham,22,6,5,11,36,41,-5,23,1.05,1838.0,Shekiera Martinez - 10,West_Ham_2024_2025,Shekiera_Martinez,10,-0.38,2024
90,10,Leicester_City,22,5,5,12,21,37,-16,20,0.91,2827.0,Janice Cayman - 4,Leicester_City_2024_2025,Janice_Cayman,4,-0.79,2024
91,11,Tottenham,22,5,5,12,26,44,-18,20,0.91,5387.0,Bethany England - 8,Tottenham_2024_2025,Bethany_England,8,-0.24,2024


In [23]:
# Save combined data
wsl_save_file_name = 'combined_wsl_data.csv'
wsl_combined.to_csv(DATA_OUT/ wsl_save_file_name, index=False)

#### Nationality Data

In [24]:
# Create a list of Datasets for merging
nationality_dfs = [
    nationality_2017_2018,
    nationality_2018_2019,
    nationality_2019_2020,
    nationality_2020_2021,
    nationality_2021_2022,
    nationality_2022_2023,
    nationality_2023_2024,
    nationality_2024_2025,
]

nationality_combined = pd.concat(nationality_dfs, ignore_index=True)

In [25]:
# Look at the combined data
nationality_combined.head(3)

,Rank,Nationality,Num_Players,Minutes_Played,List_of_Players,FIFA Ranking,season,Season_Rank,euro_flag,group,Unnamed: 6
0,1,England,150,106092.0,"Nikita Parris, Remi Allen, Ellen White, Fara W...",3,2017_2018,2017_2018_1,1,English,NaN
1,2,Wales,29,17902.0,"Rachel Rowe, Sophie Ingle, Natasha Harding, Ha...",35,2017_2018,2017_2018_2,1,European (excl. Eng),NaN
2,3,Scotland,11,11185.0,"Claire Emslie, Jennifer Beattie, Caroline Weir...",23,2017_2018,2017_2018_3,1,European (excl. Eng),NaN


In [26]:
nationality_combined = drop_columns(nationality_combined, NATIONALITY_DROP_COLUMNS)
nationality_combined = nationality_combined.drop(columns=['Unnamed: 6'])

In [27]:
# Replace Season with just one value (convert "2017_2018" → 2017)
nationality_combined['Season'] = nationality_combined['season'].str.split('_').str[0].astype(int)
nationality_combined = nationality_combined.drop(columns=['season'])

In [28]:
# Look at the data after the transformation
nationality_combined

,Rank,Nationality,Num_Players,Minutes_Played,FIFA Ranking,Season_Rank,group,Season
0,1,England,150,106092.0,3,2017_2018_1,English,2017
1,2,Wales,29,17902.0,35,2017_2018_2,European (excl. Eng),2017
2,3,Scotland,11,11185.0,23,2017_2018_3,European (excl. Eng),2017
3,4,Republic_of_Ireland,11,6344.0,29,2017_2018_4,European (excl. Eng),2017
4,5,Netherlands,10,9635.0,7,2017_2018_5,European (excl. Eng),2017
...,...,...,...,...,...,...,...,...
243,30,Chile,1,551.0,38,2024_2025_30,Non-European,2024
244,31,Hungary,1,261.0,46,2024_2025_31,European (excl. Eng),2024
245,32,Italy,1,94.0,13,2024_2025_32,European (excl. Eng),2024
246,33,Morocco,1,49.0,61,2024_2025_33,Non-European,2024


In [29]:
# Save combined data
nationality_save_file_name = 'nationality_combined_data.csv'
nationality_combined.to_csv(DATA_OUT/ nationality_save_file_name, index=False)